In [ ]:
!pip install autogluon

In [ ]:
import sys
# import sktime
import tqdm as tq
import xgboost as xgb
import matplotlib
import seaborn as sns
import sklearn as skl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import os

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [ ]:
!unzip '/content/drive/MyDrive/Colab Notebooks/elect.zip'

Archive:  /content/drive/MyDrive/Colab Notebooks/elect.zip
  inflating: building_info.csv       
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
train = pd.read_csv('./train.csv')
train.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40


In [ ]:
## 변수들을 영문명으로 변경
cols = ['num_date_time', 'num', 'date_time', 'temp', 'prec', 'wind', 'hum', 'sun', 'mj', 'power']
train.columns = cols

# 시간 관련 변수들 생성
date = pd.to_datetime(train.date_time)
train['hour'] = date.dt.hour
train['day'] = date.dt.weekday
train['month'] = date.dt.month
train['week'] = date.dt.weekofyear

#######################################
## 건물별, 요일별, 시간별 발전량 평균 넣어주기
#######################################
power_mean = pd.pivot_table(train, values = 'power', index = ['num', 'hour', 'day'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['day_hour_mean'] = train.progress_apply(lambda x : power_mean.loc[(power_mean.num == x['num']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'power'].values[0], axis = 1)

#######################################
## 건물별 시간별 발전량 평균 넣어주기
#######################################
power_hour_mean = pd.pivot_table(train, values = 'power', index = ['num', 'hour'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['hour_mean'] = train.progress_apply(lambda x : power_hour_mean.loc[(power_hour_mean.num == x['num']) & (power_hour_mean.hour == x['hour']) ,'power'].values[0], axis = 1)

#######################################
## 건물별 시간별 발전량 표준편차 넣어주기
#######################################
power_hour_std = pd.pivot_table(train, values = 'power', index = ['num', 'hour'], aggfunc = np.std).reset_index()
tqdm.pandas()
train['hour_std'] = train.progress_apply(lambda x : power_hour_std.loc[(power_hour_std.num == x['num']) & (power_hour_std.hour == x['hour']) ,'power'].values[0], axis = 1)

### 공휴일 변수 추가
train['holiday'] = train.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
train.loc[('20220601'<=train.date_time)&(train.date_time<'20220602'), 'holiday'] = 1

## https://dacon.io/competitions/official/235680/codeshare/2366?page=1&dtype=recent
train['sin_time'] = np.sin(2*np.pi*train.hour/24)
train['cos_time'] = np.cos(2*np.pi*train.hour/24)

## https://dacon.io/competitions/official/235736/codeshare/2743?page=1&dtype=recent
train['THI'] = 9/5*train['temp'] - 0.55*(1-train['hum']/100)*(9/5*train['hum']-26)+32

def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

<ipython-input-11-0531babed4e4>:10: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  train['week'] = date.dt.weekofyear
100%|██████████| 204000/204000 [01:44<00:00, 1947.58it/s]


In [ ]:
cdhs = np.array([])
for num in range(1,101):
    temp = train[train['num'] == num]
    cdh = CDH(temp['temp'].values)
    cdhs = np.concatenate([cdhs, cdh])
train['CDH'] = cdhs

In [ ]:
cols = ['num_date_time', 'num', 'date_time', 'temp', 'prec', 'wind', 'hum', 'sun', 'mj', 'power']
train.columns = cols

# 시간 관련 변수들 생성
date = pd.to_datetime(train.date_time)
train['hour'] = date.dt.hour
train['day'] = date.dt.weekday
train['month'] = date.dt.month
train['week'] = date.dt.weekofyear

train.drop(['sun', 'mj', 'num_date_time'], axis = 1, inplace = True)
train.head()

<ipython-input-8-494257d7b9a4>:16: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  train['week'] = date.dt.weekofyear


,num,date_time,temp,prec,wind,hum,power,hour,day,month,week
0,1,20220601 00,18.6,NaN,0.9,42.0,1085.28,0,2,6,22
1,1,20220601 01,18.0,NaN,1.1,45.0,1047.36,1,2,6,22
2,1,20220601 02,17.7,NaN,1.5,45.0,974.88,2,2,6,22
3,1,20220601 03,16.7,NaN,1.4,48.0,953.76,3,2,6,22
4,1,20220601 04,18.4,NaN,2.8,43.0,986.40,4,2,6,22


In [ ]:
train_y = train['power']

# 강수량 결측치 0.0으로 채우기
train['prec'].fillna(0.0, inplace=True)

# 풍속, 습도 결측치 평균으로 채우고 반올림하기
train['wind'].fillna(round(train['wind'].mean(),2), inplace=True)
train['hum'].fillna(round(train['hum'].mean(),2), inplace=True)

In [ ]:
train_y

0         1085.28
1         1047.36
2          974.88
3          953.76
4          986.40
           ...   
203995     881.04
203996     798.96
203997     825.12
203998     640.08
203999     540.24
Name: power, Length: 204000, dtype: float64

In [ ]:
train[train['THI'].isna()]

,num_date_time,num,date_time,temp,prec,wind,hum,sun,mj,power,...,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH
16643,9_20220614 11,9,20220614 11,17.6,0.0,2.13,78.72,NaN,NaN,2330.40,...,6,24,3346.640000,3215.463529,508.403756,0,0.258819,-9.659258e-01,NaN,-95.5
30161,15_20220806 17,15,20220806 17,35.6,0.0,2.13,78.72,NaN,NaN,1815.57,...,8,31,1675.758750,1754.687118,145.897501,1,-0.965926,-2.588190e-01,NaN,76.8
176477,87_20220714 05,87,20220714 05,21.9,0.0,2.13,78.72,NaN,NaN,519.66,...,7,28,622.822500,655.912588,189.435291,0,0.965926,2.588190e-01,NaN,-33.6
176478,87_20220714 06,87,20220714 06,22.3,0.0,2.13,78.72,NaN,NaN,632.88,...,7,28,754.942500,797.699647,247.673062,0,1.000000,6.123234e-17,NaN,-35.5
176479,87_20220714 07,87,20220714 07,22.8,0.0,2.13,78.72,NaN,NaN,738.72,...,7,28,917.842500,1001.745529,339.975754,0,0.965926,-2.588190e-01,NaN,-36.5
176480,87_20220714 08,87,20220714 08,23.3,0.0,2.13,78.72,NaN,NaN,1547.28,...,7,28,1562.010000,1541.318824,359.845833,0,0.866025,-5.000000e-01,NaN,-36.7
176481,87_20220714 09,87,20220714 09,23.8,0.0,2.13,78.72,NaN,NaN,2059.74,...,7,28,1864.290000,1838.949882,328.844757,0,0.707107,-7.071068e-01,NaN,-36.4
183161,90_20220806 17,90,20220806 17,35.6,0.0,2.13,78.72,NaN,NaN,2504.16,...,8,31,2368.740000,2136.875294,491.702708,1,-0.965926,-2.588190e-01,NaN,76.8
202143,100_20220608 15,100,20220608 15,24.9,0.0,2.13,78.72,NaN,NaN,880.32,...,6,23,1084.929231,1146.674824,129.975970,0,-0.707107,-7.071068e-01,NaN,-74.0


In [ ]:
train.loc[16643, 'THI'] = 9/5*train.loc[16643, 'temp'] - 0.55*(1-train.loc[16643, 'hum']/100)*(9/5*train.loc[16643, 'hum']-26)+32
train.loc[30161, 'THI'] = 9/5*train.loc[30161, 'temp'] - 0.55*(1-train.loc[30161, 'hum']/100)*(9/5*train.loc[30161, 'hum']-26)+32
train.loc[176477, 'THI'] = 9/5*train.loc[176477, 'temp'] - 0.55*(1-train.loc[176477, 'hum']/100)*(9/5*train.loc[176477, 'hum']-26)+32
train.loc[176478, 'THI'] = 9/5*train.loc[176478, 'temp'] - 0.55*(1-train.loc[176478, 'hum']/100)*(9/5*train.loc[176478, 'hum']-26)+32
train.loc[176479, 'THI'] = 9/5*train.loc[176479, 'temp'] - 0.55*(1-train.loc[176479, 'hum']/100)*(9/5*train.loc[176479, 'hum']-26)+32
train.loc[176480, 'THI'] = 9/5*train.loc[176480, 'temp'] - 0.55*(1-train.loc[176480, 'hum']/100)*(9/5*train.loc[176480, 'hum']-26)+32
train.loc[176481, 'THI'] = 9/5*train.loc[176481, 'temp'] - 0.55*(1-train.loc[176481, 'hum']/100)*(9/5*train.loc[176481, 'hum']-26)+32
train.loc[183161, 'THI'] = 9/5*train.loc[183161, 'temp'] - 0.55*(1-train.loc[183161, 'hum']/100)*(9/5*train.loc[183161, 'hum']-26)+32
train.loc[202143, 'THI'] = 9/5*train.loc[202143, 'temp'] - 0.55*(1-train.loc[202143, 'hum']/100)*(9/5*train.loc[202143, 'hum']-26)+32

In [ ]:
train.drop(['sun', 'mj', 'num_date_time', 'date_time', 'hour'], axis = 1, inplace = True)
train.head()

,num,temp,prec,wind,hum,power,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH
0,1,18.6,0.0,0.9,42.0,1085.28,2,6,22,1774.744615,1706.318118,446.882767,1,0.000000,1.000000,49.6576,-7.4
1,1,18.0,0.0,1.1,45.0,1047.36,2,6,22,1687.347692,1622.620235,439.662704,1,0.258819,0.965926,47.7625,-15.4
2,1,17.7,0.0,1.5,45.0,974.88,2,6,22,1571.483077,1506.971294,412.071906,1,0.500000,0.866025,47.2225,-23.7
3,1,16.7,0.0,1.4,48.0,953.76,2,6,22,1522.153846,1437.365647,391.205981,1,0.707107,0.707107,44.7856,-33.0
4,1,18.4,0.0,2.8,43.0,986.40,2,6,22,1506.793846,1447.321412,381.099697,1,0.866025,0.500000,49.0061,-40.6


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204000 entries, 0 to 203999
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   num            204000 non-null  int64  
 1   temp           204000 non-null  float64
 2   prec           204000 non-null  float64
 3   wind           204000 non-null  float64
 4   hum            204000 non-null  float64
 5   power          204000 non-null  float64
 6   day            204000 non-null  int64  
 7   month          204000 non-null  int64  
 8   week           204000 non-null  int64  
 9   day_hour_mean  204000 non-null  float64
 10  hour_mean      204000 non-null  float64
 11  hour_std       204000 non-null  float64
 12  holiday        204000 non-null  int64  
 13  sin_time       204000 non-null  float64
 14  cos_time       204000 non-null  float64
 15  THI            204000 non-null  float64
 16  CDH            204000 non-null  float64
dtypes: float64(12), int64(5)
memo

In [ ]:
test = pd.read_csv('./test.csv')
test.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%)
0,1_20220825 00,1,20220825 00,23.5,0.0,2.2,72
1,1_20220825 01,1,20220825 01,23.0,0.0,0.9,72
2,1_20220825 02,1,20220825 02,22.7,0.0,1.5,75
3,1_20220825 03,1,20220825 03,22.1,0.0,1.3,78
4,1_20220825 04,1,20220825 04,21.8,0.0,1.0,77


In [ ]:
# train set과 동일한 전처리 과정
cols = ['num_date_time', 'num', 'date_time', 'temp', 'prec', 'wind','hum']
test.columns = cols
date = pd.to_datetime(test.date_time)
test['hour'] = date.dt.hour
test['day'] = date.dt.weekday
test['month'] = date.dt.month
test['week'] = date.dt.weekofyear
test['sin_time'] = np.sin(2*np.pi*test.hour/24)
test['cos_time'] = np.cos(2*np.pi*test.hour/24)
test['holiday'] = test.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)

## 건물별 일별 시간별 발전량 평균
tqdm.pandas()
test['day_hour_mean'] = test.progress_apply(lambda x : power_mean.loc[(power_mean.num == x['num']) & (power_mean.day == x['day']) & (power_mean.hour == x['hour']) ,'power'].values[0], axis = 1)

## 건물별 시간별 발전량 평균 넣어주기
tqdm.pandas()
test['hour_mean'] = test.progress_apply(lambda x : power_hour_mean.loc[(power_hour_mean.num == x['num']) & (power_hour_mean.hour == x['hour']) ,'power'].values[0], axis = 1)

tqdm.pandas()
test['hour_std'] = test.progress_apply(lambda x : power_hour_std.loc[(power_hour_std.num == x['num']) & (power_hour_std.hour == x['hour']) ,'power'].values[0], axis = 1)

test.drop(['hour','date_time', 'num_date_time'], axis = 1, inplace = True)

# pandas 내 선형보간 method 사용
for i in range(100):
    test.iloc[i*168:(i+1)*168, :]  = test.iloc[i*168:(i+1)*168, :].interpolate()


test['THI'] = 9/5*test['temp'] - 0.55*(1-test['hum']/100)*(9/5*test['hum']-26)+32

cdhs = np.array([])
for num in range(1,101):
    temp = test[test['num'] == num]
    cdh = CDH(temp['temp'].values)
    cdhs = np.concatenate([cdhs, cdh])
test['CDH'] = cdhs

test = test[['num','temp', 'prec', 'wind', 'hum', 'day', 'month', 'week',
       'day_hour_mean', 'hour_mean', 'hour_std', 'holiday', 'sin_time',
       'cos_time', 'THI', 'CDH']]
test.head()

<ipython-input-20-8c9ac5c44cf7>:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  test['week'] = date.dt.weekofyear
100%|██████████| 16800/16800 [00:08<00:00, 1889.02it/s]


,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH
0,1,23.5,0.0,2.2,72,3,8,34,1627.80,1706.318118,446.882767,0,0.000000,1.000000,58.3456,-2.5
1,1,23.0,0.0,0.9,72,3,8,34,1550.08,1622.620235,439.662704,0,0.258819,0.965926,57.4456,-5.5
2,1,22.7,0.0,1.5,75,3,8,34,1431.12,1506.971294,412.071906,0,0.500000,0.866025,57.8725,-8.8
3,1,22.1,0.0,1.3,78,3,8,34,1372.20,1437.365647,391.205981,0,0.707107,0.707107,57.9376,-12.7
4,1,21.8,0.0,1.0,77,3,8,34,1381.72,1447.321412,381.099697,0,0.866025,0.500000,56.9961,-16.9


In [ ]:
cols = ['num_date_time', 'num', 'date_time', 'temp', 'prec', 'wind','hum']
test.columns = cols
date = pd.to_datetime(test.date_time)
test['hour'] = date.dt.hour
test['day'] = date.dt.weekday
test['month'] = date.dt.month
test['week'] = date.dt.weekofyear
test.drop(['num_date_time'], axis = 1, inplace = True)
test.head()

<ipython-input-11-d33920a13f53>:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  test['week'] = date.dt.weekofyear


,num,date_time,temp,prec,wind,hum,hour,day,month,week
0,1,20220825 00,23.5,0.0,2.2,72,0,3,8,34
1,1,20220825 01,23.0,0.0,0.9,72,1,3,8,34
2,1,20220825 02,22.7,0.0,1.5,75,2,3,8,34
3,1,20220825 03,22.1,0.0,1.3,78,3,3,8,34
4,1,20220825 04,21.8,0.0,1.0,77,4,3,8,34


In [ ]:
train.to_csv('/content/drive/MyDrive/Colab Notebooks/train_origin.csv', index = False)
test.to_csv('/content/drive/MyDrive/Colab Notebooks/test_origin.csv', index = False)

NameError: ignored

In [ ]:
train.to_csv('/content/drive/MyDrive/Colab Notebooks/train_pre.csv', index = False)
test.to_csv('/content/drive/MyDrive/Colab Notebooks/test_pre.csv', index = False)

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16800 entries, 0 to 16799
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   num            16800 non-null  int64  
 1   temp           16800 non-null  float64
 2   prec           16800 non-null  float64
 3   wind           16800 non-null  float64
 4   hum            16800 non-null  int64  
 5   day            16800 non-null  int64  
 6   month          16800 non-null  int64  
 7   week           16800 non-null  int64  
 8   day_hour_mean  16800 non-null  float64
 9   hour_mean      16800 non-null  float64
 10  hour_std       16800 non-null  float64
 11  holiday        16800 non-null  int64  
 12  sin_time       16800 non-null  float64
 13  cos_time       16800 non-null  float64
 14  THI            16800 non-null  float64
 15  CDH            16800 non-null  float64
dtypes: float64(10), int64(6)
memory usage: 2.1 MB


In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
# 스케일링
import numpy as np
from sklearn.preprocessing import QuantileTransformer

# Quantile Transformer 객체 생성
# output_distribution은 'uniform' 또는 'normal' 중 선택 가능합니다.
quantile_transformer = QuantileTransformer(output_distribution='normal')

# Quantile Transformation 수행
transformed_train = quantile_transformer.fit_transform(train)
transformed_test = quantile_transformer.fit_transform(test)

In [ ]:
transformed_train = pd.DataFrame(data = transformed_train, columns = train.columns)
transformed_train.head()

,num,temp,prec,wind,hum,power,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH
0,-5.199338,-1.620592,-5.199338,-0.945616,-2.119668,-0.680199,-0.374936,-5.199338,-5.199338,-0.018822,-0.091712,0.663856,5.199338,-0.069056,5.199338,-1.943681,-0.196837
1,-5.199338,-1.761948,-5.199338,-0.752997,-1.976943,-0.736147,-0.374936,-5.199338,-5.199338,-0.079593,-0.157469,0.630254,5.199338,0.140977,1.529558,-2.211406,-0.389783
2,-5.199338,-1.837975,-5.199338,-0.387078,-1.976943,-0.844448,-0.374936,-5.199338,-5.199338,-0.172174,-0.256964,0.540556,5.199338,0.407442,1.055293,-2.270733,-0.607463
3,-5.199338,-2.107953,-5.199338,-0.475214,-1.805023,-0.886730,-0.374936,-5.199338,-5.199338,-0.225732,-0.328478,0.497809,5.199338,0.590949,0.743040,-2.511791,-0.824933
4,-5.199338,-1.664082,-5.199338,0.648724,-2.063775,-0.830234,-0.374936,-5.199338,-5.199338,-0.239660,-0.315638,0.473821,5.199338,0.866189,0.497809,-2.044742,-0.995402


In [ ]:
transformed_test = pd.DataFrame(data = transformed_test, columns = test.columns)
transformed_test.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH
0,-5.199338,0.622621,-5.199338,0.401995,-0.411534,0.010037,-5.199338,-5.199338,-0.107675,-0.094948,0.651580,-5.199338,-0.045180,5.199338,-0.374936,1.550085
1,-5.199338,0.457033,-5.199338,-0.758004,-0.411534,0.010037,-5.199338,-5.199338,-0.170523,-0.154981,0.621099,-5.199338,0.162552,1.529558,-0.542335,1.362078
2,-5.199338,0.345485,-5.199338,-0.177827,-0.251016,0.010037,-5.199338,-5.199338,-0.291252,-0.262136,0.523204,-5.199338,0.427976,1.053106,-0.468203,1.228607
3,-5.199338,0.139710,-5.199338,-0.342823,-0.080379,0.010037,-5.199338,-5.199338,-0.347825,-0.334050,0.475214,-5.199338,0.605955,0.734798,-0.450080,1.079690
4,-5.199338,0.035135,-5.199338,-0.641002,-0.138444,0.010037,-5.199338,-5.199338,-0.336331,-0.320290,0.452077,-5.199338,0.879040,0.489307,-0.598435,0.947580


In [ ]:
transformed_train['power'] = train_y
transformed_train.head()

,num,temp,prec,wind,hum,power,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH
0,-5.199338,-1.620592,-5.199338,-0.945616,-2.119668,1085.28,-0.374936,-5.199338,-5.199338,-0.018822,-0.091712,0.663856,5.199338,-0.069056,5.199338,-1.943681,-0.196837
1,-5.199338,-1.761948,-5.199338,-0.752997,-1.976943,1047.36,-0.374936,-5.199338,-5.199338,-0.079593,-0.157469,0.630254,5.199338,0.140977,1.529558,-2.211406,-0.389783
2,-5.199338,-1.837975,-5.199338,-0.387078,-1.976943,974.88,-0.374936,-5.199338,-5.199338,-0.172174,-0.256964,0.540556,5.199338,0.407442,1.055293,-2.270733,-0.607463
3,-5.199338,-2.107953,-5.199338,-0.475214,-1.805023,953.76,-0.374936,-5.199338,-5.199338,-0.225732,-0.328478,0.497809,5.199338,0.590949,0.743040,-2.511791,-0.824933
4,-5.199338,-1.664082,-5.199338,0.648724,-2.063775,986.40,-0.374936,-5.199338,-5.199338,-0.239660,-0.315638,0.473821,5.199338,0.866189,0.497809,-2.044742,-0.995402


In [ ]:
transformed_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204000 entries, 0 to 203999
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   num            204000 non-null  float64
 1   temp           204000 non-null  float64
 2   prec           204000 non-null  float64
 3   wind           204000 non-null  float64
 4   hum            204000 non-null  float64
 5   power          204000 non-null  float64
 6   day            204000 non-null  float64
 7   month          204000 non-null  float64
 8   week           204000 non-null  float64
 9   day_hour_mean  204000 non-null  float64
 10  hour_mean      204000 non-null  float64
 11  hour_std       204000 non-null  float64
 12  holiday        204000 non-null  float64
 13  sin_time       204000 non-null  float64
 14  cos_time       204000 non-null  float64
 15  THI            204000 non-null  float64
 16  CDH            204000 non-null  float64
dtypes: float64(17)
memory usage: 

In [ ]:
transformed_train.to_csv('./train_final_e.csv', index = False)
transformed_test.to_csv('./test_final_e.csv', index = False)

In [ ]:
train_data = TabularDataset('./train_pre.csv')
test_data = TabularDataset('./test_pre.csv')

label = 'power'
eval_metric = 'mape'
time_limit = 3600 * 6

predictor = TabularPredictor(
    label=label, eval_metric=eval_metric
).fit(train_data, presets='best_quality', time_limit=time_limit, num_gpus=1)
predictor.leaderboard(silent = True)

No path specified. Models will be saved in: "AutogluonModels/ag-20230725_013123/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 21600s
AutoGluon will save models to "AutogluonModels/ag-20230725_013123/"
AutoGluon Version:  0.8.2
Python Version:     3.10.6
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Jun 9 10:57:30 UTC 2023
Disk Space Avail:   146.64 GB / 179.07 GB (81.9%)
Train Data Rows:    204000
Train Data Columns: 16
Label Column: power
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (25488.4, 0.0, 2451.03646, 2440.64886)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_ty

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost_BAG_L2,-2.606207e+12,3250.808096,11607.919902,16.438748,232.413800,2,True,19
1,WeightedEnsemble_L3,-2.606207e+12,3250.812562,11611.237819,0.004466,3.317917,3,True,22
2,XGBoost_BAG_L1,-2.858482e+12,173.532662,1848.983034,173.532662,1848.983034,1,True,9
3,WeightedEnsemble_L2,-2.858482e+12,173.536771,1852.596824,0.004109,3.613791,2,True,12
4,KNeighborsDist_BAG_L1,-2.356608e+13,46.521423,0.939082,46.521423,0.939082,1,True,2
5,NeuralNetTorch_BAG_L2,-2.368941e+13,3237.717172,12772.394275,3.347824,1396.888173,2,True,20
6,KNeighborsUnif_BAG_L1,-2.559104e+13,46.865713,0.980661,46.865713,0.980661,1,True,1
7,LightGBMXT_BAG_L2,-2.604732e+13,4000.010871,12078.380819,765.641523,702.874717,2,True,13
8,LightGBMLarge_BAG_L2,-2.648680e+13,3741.634285,12189.570172,507.264937,814.064070,2,True,21
9,RandomForestMSE_BAG_L1,-2.653090e+13,5.892527,63.014307,5.892527,63.014307,1,True,5


In [ ]:
model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_data, model=model_to_use)
submission = pd.read_csv('/content/sample_submission.csv')
submission['answer'] = model_pred
submission.head()

,num_date_time,answer
0,1_20220825 00,1205.336426
1,1_20220825 01,1155.597778
2,1_20220825 02,1131.238281
3,1_20220825 03,1092.015259
4,1_20220825 04,1104.078491


In [ ]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/20230725_1_e.csv', index=False)

In [ ]:
!pip install autogluon

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
train_data = TabularDataset('/content/drive/MyDrive/Colab Notebooks/train_origin.csv')
test_data = TabularDataset('/content/drive/MyDrive/Colab Notebooks/test_origin.csv')

label = 'power'
eval_metric = 'mape'
time_limit = 3600 * 3

predictor = TabularPredictor(
    label=label, eval_metric=eval_metric
).fit(train_data, presets='best_quality', time_limit=time_limit, num_gpus=1)

predictor.leaderboard(silent = True)

No path specified. Models will be saved in: "AutogluonModels/ag-20230726_002709/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 10800s
AutoGluon will save models to "AutogluonModels/ag-20230726_002709/"
AutoGluon Version:  0.8.2
Python Version:     3.10.6
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Jun 9 10:57:30 UTC 2023
Disk Space Avail:   146.72 GB / 179.07 GB (81.9%)
Train Data Rows:    204000
Train Data Columns: 10
Label Column: power
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (25488.4, 0.0, 2451.03646, 2440.64886)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_ty

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost_BAG_L2,-1.755382e+12,1479.955556,4809.624703,9.085795,78.108880,2,True,19
1,WeightedEnsemble_L3,-1.755382e+12,1479.959603,4812.698190,0.004048,3.073487,3,True,22
2,XGBoost_BAG_L1,-4.020603e+12,69.126474,299.561507,69.126474,299.561507,1,True,9
3,WeightedEnsemble_L2,-4.020603e+12,69.130697,303.340194,0.004223,3.778687,2,True,12
4,LightGBMLarge_BAG_L2,-1.669924e+13,1729.575281,5028.139908,258.705521,296.624085,2,True,21
5,RandomForestMSE_BAG_L2,-1.691406e+13,1477.854478,4964.204093,6.984718,232.688270,2,True,15
6,CatBoost_BAG_L2,-1.710603e+13,1471.317289,4972.634045,0.447529,241.118222,2,True,16
7,LightGBMXT_BAG_L2,-1.719542e+13,1845.415176,4987.355911,374.545416,255.840088,2,True,13
8,ExtraTreesMSE_BAG_L2,-1.740168e+13,1477.354527,4760.215962,6.484767,28.700139,2,True,17
9,LightGBMLarge_BAG_L1,-1.754242e+13,600.787979,203.749515,600.787979,203.749515,1,True,11


In [ ]:
submission = pd.read_csv('/content/sample_submission.csv')
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/submission.csv', index = False)

In [ ]:
model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_data, model=model_to_use)
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/submission.csv')
submission['answer'] = model_pred
submission.head()

,num_date_time,answer
0,1_20220825 00,1831.676636
1,1_20220825 01,1803.876465
2,1_20220825 02,1663.724609
3,1_20220825 03,1564.810791
4,1_20220825 04,1453.684082


In [ ]:
submission.to_csv('./20230726_1.csv', index = False) # 10.5526718162: 전처리 없음 + autogluon

In [ ]:
train_data = TabularDataset('/content/drive/MyDrive/Colab Notebooks/train_pre.csv')
test_data = TabularDataset('/content/drive/MyDrive/Colab Notebooks/test_pre.csv')

label = 'power'
eval_metric = 'mape'
time_limit = 3600 * 3

predictor = TabularPredictor(
    label=label, eval_metric=eval_metric
).fit(train_data, presets='best_quality', time_limit=time_limit, num_gpus=1)

predictor.leaderboard(silent = True)

Loaded data from: /content/drive/MyDrive/Colab Notebooks/train_pre.csv | Columns = 17 / 17 | Rows = 204000 -> 204000
Loaded data from: /content/drive/MyDrive/Colab Notebooks/test_pre.csv | Columns = 16 / 16 | Rows = 16800 -> 16800
No path specified. Models will be saved in: "AutogluonModels/ag-20230726_030037/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 10800s
AutoGluon will save models to "AutogluonModels/ag-20230726_030037/"
AutoGluon Version:  0.8.2
Python Version:     3.10.6
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Jun 9 10:57:30 UTC 2023
Disk Space Avail:   141.34 GB / 179.07 GB (78.9%)
Train Data Rows:    204000
Train Data Columns: 16
Label Column: power
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observe

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost_BAG_L2,-2.342370e+12,2602.204745,5256.315007,4.830251,57.393719,2,True,19
1,WeightedEnsemble_L3,-2.342370e+12,2602.209198,5259.512359,0.004452,3.197352,3,True,22
2,XGBoost_BAG_L1,-2.862596e+12,144.913743,691.678838,144.913743,691.678838,1,True,9
3,WeightedEnsemble_L2,-2.862596e+12,144.917672,695.387567,0.003929,3.708728,2,True,12
4,ExtraTreesMSE_BAG_L2,-1.989150e+13,2604.436517,5237.910100,7.062022,38.988812,2,True,17
5,RandomForestMSE_BAG_L2,-2.074431e+13,2605.762103,5546.439240,8.387609,347.517951,2,True,15
6,CatBoost_BAG_L2,-2.085645e+13,2597.762386,5388.615682,0.387892,189.694393,2,True,16
7,LightGBMLarge_BAG_L2,-2.104476e+13,2860.966361,5523.182157,263.591866,324.260869,2,True,21
8,LightGBM_BAG_L2,-2.110812e+13,2663.918201,5396.096803,66.543707,197.175515,2,True,14
9,LightGBMXT_BAG_L2,-2.179635e+13,3107.140652,5490.667743,509.766158,291.746455,2,True,13


In [ ]:
model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_data, model=model_to_use)
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/submission.csv')
submission['answer'] = model_pred
submission.head()

,num_date_time,answer
0,1_20220825 00,1719.796875
1,1_20220825 01,1733.634766
2,1_20220825 02,1550.221191
3,1_20220825 03,1525.985840
4,1_20220825 04,1577.103882


In [ ]:
submission.to_csv('./20230726_2.csv', index = False) # : 전처리(컬럼 추가) + autogluon

In [ ]:
! pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.3 MB/s eta 0:00:00


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_origin.csv')
train_y = train['power']
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_origin.csv')
train_y.shape

(204000,)

In [ ]:
train.head()

,num,date_time,temp,prec,wind,hum,power,hour,day,month,week
0,1,20220601 00,18.6,0.0,0.9,42.0,1085.28,0,2,6,22
1,1,20220601 01,18.0,0.0,1.1,45.0,1047.36,1,2,6,22
2,1,20220601 02,17.7,0.0,1.5,45.0,974.88,2,2,6,22
3,1,20220601 03,16.7,0.0,1.4,48.0,953.76,3,2,6,22
4,1,20220601 04,18.4,0.0,2.8,43.0,986.40,4,2,6,22


In [ ]:
train.drop(['date_time', 'power'], axis = 1, inplace = True)
test.drop('date_time', axis = 1, inplace = True)

In [ ]:
def SMAPE(true, pred):
    v = 2 * abs(pred - true) / (abs(pred) + abs(true))
    output = np.mean(v) * 100
    return output

In [ ]:
import optuna
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split

# 데이터 로드 및 전처리 (X_train, X_valid, y_train, y_valid)

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 1000, 100)
    max_depth = trial.suggest_int('max_depth', 10, 30, 5)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20, 2)
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])


    model = ExtraTreesRegressor(
        n_estimators=n_estimators,
        min_samples_split=min_samples_split,
        max_depth=max_depth,
        bootstrap=bootstrap
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    score = SMAPE(y_valid, preds)
    return score

X_train, X_valid, y_train, y_valid = train_test_split(train, train_y, test_size = 0.2)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300)

print('Best trial:', study.best_trial.params)
print('Best score:', study.best_value)

[I 2023-07-26 05:34:37,210] A new study created in memory with name: no-name-92198d2d-956a-4452-82a6-016a7660d3a0
[I 2023-07-26 05:36:03,661] Trial 0 finished with value: 0.0028440282253553517 and parameters: {'n_estimators': 400, 'max_depth': 30, 'min_samples_split': 10, 'bootstrap': True}. Best is trial 0 with value: 0.0028440282253553517.
[I 2023-07-26 05:37:21,732] Trial 1 finished with value: 0.0026823319603697876 and parameters: {'n_estimators': 300, 'max_depth': 30, 'min_samples_split': 4, 'bootstrap': True}. Best is trial 1 with value: 0.0026823319603697876.
[I 2023-07-26 05:38:11,707] Trial 2 finished with value: 0.014420526825906659 and parameters: {'n_estimators': 200, 'max_depth': 20, 'min_samples_split': 16, 'bootstrap': False}. Best is trial 1 with value: 0.0026823319603697876.
[I 2023-07-26 05:40:43,914] Trial 3 finished with value: 0.003028273510615964 and parameters: {'n_estimators': 700, 'max_depth': 25, 'min_samples_split': 8, 'bootstrap': True}. Best is trial 1 with

KeyboardInterrupt: ignored

In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 800, 1000, 10)
    max_depth = trial.suggest_int('max_depth', 10, 30, 5)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20, 2)

    model = ExtraTreesRegressor(
        n_estimators=n_estimators,
        min_samples_split=min_samples_split,
        max_depth=max_depth,
        bootstrap=False
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    score = SMAPE(y_valid, preds)
    return score

X_train, X_valid, y_train, y_valid = train_test_split(train, train_y, test_size = 0.2)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300)

print('Best trial:', study.best_trial.params)
print('Best score:', study.best_value)

[I 2023-07-26 07:45:39,918] A new study created in memory with name: no-name-e7510b31-b122-4617-bc1e-883f296ea4bf
[I 2023-07-26 07:52:38,577] Trial 0 finished with value: 0.0012568018160840238 and parameters: {'n_estimators': 980, 'max_depth': 30, 'min_samples_split': 2}. Best is trial 0 with value: 0.0012568018160840238.
[I 2023-07-26 07:55:30,433] Trial 1 finished with value: 0.05872304116474544 and parameters: {'n_estimators': 830, 'max_depth': 15, 'min_samples_split': 14}. Best is trial 0 with value: 0.0012568018160840238.
[I 2023-07-26 07:59:57,729] Trial 2 finished with value: 0.007558531800315315 and parameters: {'n_estimators': 880, 'max_depth': 20, 'min_samples_split': 2}. Best is trial 0 with value: 0.0012568018160840238.
[I 2023-07-26 08:05:07,763] Trial 3 finished with value: 0.0022626242026975536 and parameters: {'n_estimators': 960, 'max_depth': 25, 'min_samples_split': 10}. Best is trial 0 with value: 0.0012568018160840238.
[I 2023-07-26 08:07:09,940] Trial 4 finished wi

KeyboardInterrupt: ignored

In [ ]:
print('Best trial:', study.best_trial.params)
print('Best score:', study.best_value)

Best trial: {'n_estimators': 880, 'max_depth': 30, 'min_samples_split': 2}
Best score: 0.0012361614065072748


In [ ]:
params ={'n_estimators': 880, 'max_depth': 30, 'min_samples_split': 2, 'bootstrap': False}
model = ExtraTreesRegressor(**params)
model.fit(train, train_y)

# 테스트 데이터에 대한 예측
preds = model.predict(test)

submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/submission.csv')
submission['answer'] = preds
submission.head()

ValueError: ignored

In [ ]:
submission.to_csv('./20230726_3.csv', index=False)

In [ ]:
train.head()

,num,temp,prec,wind,hum,hour,day,month,week
0,1,18.6,0.0,0.9,42.0,0,2,6,22
1,1,18.0,0.0,1.1,45.0,1,2,6,22
2,1,17.7,0.0,1.5,45.0,2,2,6,22
3,1,16.7,0.0,1.4,48.0,3,2,6,22
4,1,18.4,0.0,2.8,43.0,4,2,6,22


In [ ]:
test.head()

,num,temp,prec,wind,hum,hour,day,month,week
0,1,23.5,0.0,2.2,72,0,3,8,34
1,1,23.0,0.0,0.9,72,1,3,8,34
2,1,22.7,0.0,1.5,75,2,3,8,34
3,1,22.1,0.0,1.3,78,3,3,8,34
4,1,21.8,0.0,1.0,77,4,3,8,34


In [ ]:
train_y.head()

0    1085.28
1    1047.36
2     974.88
3     953.76
4     986.40
Name: power, dtype: float64

In [ ]:
import optuna
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split

# 데이터 로드 및 전처리 (X_train, X_valid, y_train, y_valid)

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 1000, 100)
    max_depth = trial.suggest_int('max_depth', 10, 30, 5)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20, 2)
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])


    model = ExtraTreesRegressor(
        n_estimators=n_estimators,
        min_samples_split=min_samples_split,
        max_depth=max_depth,
        bootstrap=bootstrap
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    score = SMAPE(y_valid, preds)
    return score

X_train, X_valid, y_train, y_valid = train_test_split(train, train_y, test_size = 0.2)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300)

print('Best trial:', study.best_trial.params)
print('Best score:', study.best_value)

[I 2023-07-27 00:33:58,098] A new study created in memory with name: no-name-fd177998-f6ce-4691-96a4-73cf9c22a741
[I 2023-07-27 00:36:26,337] Trial 0 finished with value: 6.781412662700303 and parameters: {'n_estimators': 700, 'max_depth': 25, 'min_samples_split': 20, 'bootstrap': False}. Best is trial 0 with value: 6.781412662700303.
[I 2023-07-27 00:38:56,678] Trial 1 finished with value: 6.483361727568135 and parameters: {'n_estimators': 800, 'max_depth': 25, 'min_samples_split': 6, 'bootstrap': True}. Best is trial 1 with value: 6.483361727568135.
[I 2023-07-27 00:40:24,432] Trial 2 finished with value: 6.771094976387918 and parameters: {'n_estimators': 500, 'max_depth': 30, 'min_samples_split': 8, 'bootstrap': True}. Best is trial 1 with value: 6.483361727568135.
[I 2023-07-27 00:40:50,860] Trial 3 finished with value: 12.735051034633754 and parameters: {'n_estimators': 200, 'max_depth': 15, 'min_samples_split': 4, 'bootstrap': True}. Best is trial 1 with value: 6.483361727568135.

KeyboardInterrupt: ignored

In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 800, 1000, 10)
    max_depth = trial.suggest_int('max_depth', 10, 30, 5)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20, 2)

    model = ExtraTreesRegressor(
        n_estimators=n_estimators,
        min_samples_split=min_samples_split,
        max_depth=max_depth,
        bootstrap=True
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    score = SMAPE(y_valid, preds)
    return score

X_train, X_valid, y_train, y_valid = train_test_split(train, train_y, test_size = 0.2)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300)

[I 2023-07-27 01:14:34,267] A new study created in memory with name: no-name-6c2e1e11-2bd3-45b3-b0d7-fff03d36c029
[I 2023-07-27 01:17:22,609] Trial 0 finished with value: 6.929971541669756 and parameters: {'n_estimators': 890, 'max_depth': 20, 'min_samples_split': 4}. Best is trial 0 with value: 6.929971541669756.
[I 2023-07-27 01:21:36,082] Trial 1 finished with value: 6.003450708574113 and parameters: {'n_estimators': 970, 'max_depth': 25, 'min_samples_split': 2}. Best is trial 1 with value: 6.003450708574113.
[I 2023-07-27 01:24:42,534] Trial 2 finished with value: 6.769141081743007 and parameters: {'n_estimators': 870, 'max_depth': 20, 'min_samples_split': 2}. Best is trial 1 with value: 6.003450708574113.
[I 2023-07-27 01:26:38,390] Trial 3 finished with value: 7.8980683533714 and parameters: {'n_estimators': 800, 'max_depth': 25, 'min_samples_split': 16}. Best is trial 1 with value: 6.003450708574113.
[I 2023-07-27 01:28:43,267] Trial 4 finished with value: 7.320634069615926 and 

KeyboardInterrupt: ignored

In [ ]:
print('Best trial:', study.best_trial.params)
print('Best score:', study.best_value)

Best trial: {'n_estimators': 880, 'max_depth': 30, 'min_samples_split': 2}
Best score: 5.896064555348294


In [ ]:
params ={'n_estimators': 880, 'max_depth': 30, 'min_samples_split': 2, 'bootstrap': True}
model = ExtraTreesRegressor(**params)
model.fit(train, train_y)

# 테스트 데이터에 대한 예측
preds = model.predict(test)

submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/submission.csv')
submission['answer'] = preds
submission.head()

,num_date_time,answer
0,1_20220825 00,2072.556136
1,1_20220825 01,2040.548045
2,1_20220825 02,1928.032636
3,1_20220825 03,1892.237182
4,1_20220825 04,1881.268364


In [ ]:
submission.to_csv('./전처리 안 함 + optuna.csv', index=False)

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_pre.csv')
train_y = train['power']
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_pre.csv')
train_y.shape

(204000,)

In [ ]:
train.head()

,num,temp,prec,wind,hum,power,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH
0,1,18.6,0.0,0.9,42.0,1085.28,2,6,22,1774.744615,1706.318118,446.882767,1,0.000000,1.000000,49.6576,-7.4
1,1,18.0,0.0,1.1,45.0,1047.36,2,6,22,1687.347692,1622.620235,439.662704,1,0.258819,0.965926,47.7625,-15.4
2,1,17.7,0.0,1.5,45.0,974.88,2,6,22,1571.483077,1506.971294,412.071906,1,0.500000,0.866025,47.2225,-23.7
3,1,16.7,0.0,1.4,48.0,953.76,2,6,22,1522.153846,1437.365647,391.205981,1,0.707107,0.707107,44.7856,-33.0
4,1,18.4,0.0,2.8,43.0,986.40,2,6,22,1506.793846,1447.321412,381.099697,1,0.866025,0.500000,49.0061,-40.6


In [ ]:
test.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH
0,1,23.5,0.0,2.2,72,3,8,34,1627.80,1706.318118,446.882767,0,0.000000,1.000000,58.3456,-2.5
1,1,23.0,0.0,0.9,72,3,8,34,1550.08,1622.620235,439.662704,0,0.258819,0.965926,57.4456,-5.5
2,1,22.7,0.0,1.5,75,3,8,34,1431.12,1506.971294,412.071906,0,0.500000,0.866025,57.8725,-8.8
3,1,22.1,0.0,1.3,78,3,8,34,1372.20,1437.365647,391.205981,0,0.707107,0.707107,57.9376,-12.7
4,1,21.8,0.0,1.0,77,3,8,34,1381.72,1447.321412,381.099697,0,0.866025,0.500000,56.9961,-16.9


In [ ]:
train.drop(['power'], axis = 1, inplace = True)

In [ ]:
def SMAPE(true, pred):
    v = 2 * abs(pred - true) / (abs(pred) + abs(true))
    output = np.mean(v) * 100
    return output

In [ ]:
import optuna
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split

# 데이터 로드 및 전처리 (X_train, X_valid, y_train, y_valid)

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 1000, 100)
    max_depth = trial.suggest_int('max_depth', 10, 30, 5)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20, 2)
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])


    model = ExtraTreesRegressor(
        n_estimators=n_estimators,
        min_samples_split=min_samples_split,
        max_depth=max_depth,
        bootstrap=bootstrap
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    score = SMAPE(y_valid, preds)
    return score

X_train, X_valid, y_train, y_valid = train_test_split(train, train_y, test_size = 0.2)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300)

[I 2023-07-27 04:32:26,965] A new study created in memory with name: no-name-8f7eb6cb-7a58-45a0-9e0b-9230d09817b7
[I 2023-07-27 04:33:07,767] Trial 0 finished with value: 4.720409449170732 and parameters: {'n_estimators': 100, 'max_depth': 30, 'min_samples_split': 20, 'bootstrap': False}. Best is trial 0 with value: 4.720409449170732.
[I 2023-07-27 04:34:29,733] Trial 1 finished with value: 4.830789075914862 and parameters: {'n_estimators': 300, 'max_depth': 20, 'min_samples_split': 8, 'bootstrap': True}. Best is trial 0 with value: 4.720409449170732.
[I 2023-07-27 04:35:04,063] Trial 2 finished with value: 4.563299777386912 and parameters: {'n_estimators': 100, 'max_depth': 30, 'min_samples_split': 6, 'bootstrap': True}. Best is trial 2 with value: 4.563299777386912.
[I 2023-07-27 04:36:59,468] Trial 3 finished with value: 6.9222481226255725 and parameters: {'n_estimators': 900, 'max_depth': 10, 'min_samples_split': 4, 'bootstrap': True}. Best is trial 2 with value: 4.563299777386912.

KeyboardInterrupt: ignored

In [ ]:
print('Best trial:', study.best_trial.params)
print('Best score:', study.best_value)

Best trial: {'n_estimators': 400, 'max_depth': 30, 'min_samples_split': 2, 'bootstrap': False}
Best score: 4.1019593786434525


In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 400, 600, 10)
    max_depth = trial.suggest_int('max_depth', 25, 30, 1)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 5, 1)
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])

    model = ExtraTreesRegressor(
        n_estimators=n_estimators,
        min_samples_split=min_samples_split,
        max_depth=max_depth,
        bootstrap=bootstrap
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    score = SMAPE(y_valid, preds)
    return score

X_train, X_valid, y_train, y_valid = train_test_split(train, train_y, test_size = 0.2)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300)

[I 2023-07-27 07:17:14,717] A new study created in memory with name: no-name-656b0636-cb5e-42d7-9dbf-d2d2ee658e53
[I 2023-07-27 07:23:40,598] Trial 0 finished with value: 4.157182165150307 and parameters: {'n_estimators': 580, 'max_depth': 29, 'min_samples_split': 2, 'bootstrap': False}. Best is trial 0 with value: 4.157182165150307.
[I 2023-07-27 07:26:23,430] Trial 1 finished with value: 4.478217422112483 and parameters: {'n_estimators': 410, 'max_depth': 28, 'min_samples_split': 3, 'bootstrap': True}. Best is trial 0 with value: 4.157182165150307.
[I 2023-07-27 07:30:33,977] Trial 2 finished with value: 4.477798812248445 and parameters: {'n_estimators': 590, 'max_depth': 26, 'min_samples_split': 2, 'bootstrap': True}. Best is trial 0 with value: 4.157182165150307.
[I 2023-07-27 07:34:20,937] Trial 3 finished with value: 4.48328263093263 and parameters: {'n_estimators': 580, 'max_depth': 27, 'min_samples_split': 3, 'bootstrap': True}. Best is trial 0 with value: 4.157182165150307.
[I

KeyboardInterrupt: ignored

In [ ]:
print('Best trial:', study.best_trial.params)
print('Best score:', study.best_value)

Best trial: {'n_estimators': 580, 'max_depth': 29, 'min_samples_split': 2, 'bootstrap': False}
Best score: 4.157182165150307


In [ ]:
params ={'n_estimators': 400, 'max_depth': 30, 'min_samples_split': 2, 'bootstrap': False}
model = ExtraTreesRegressor(**params)
model.fit(train, train_y)

# 테스트 데이터에 대한 예측
preds = model.predict(test)

submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/submission.csv')
submission['answer'] = preds
submission.head()

,num_date_time,answer
0,1_20220825 00,1849.238725
1,1_20220825 01,1826.953689
2,1_20220825 02,1553.387975
3,1_20220825 03,1488.119321
4,1_20220825 04,1506.048083


In [ ]:
submission.to_csv('./전처리함 + optuna.csv', index=False)